Celda 1 – Encabezado y librerías

In [ ]:
"""
Proyecto Módulo 4 - Preparación de datos con Python
Flujo completo: NumPy + Pandas + obtención, limpieza, wrangling y agrupamiento.
"""

import numpy as np
import pandas as pd


Celda 2 – Lección 1: NumPy

In [ ]:
# LECCIÓN 1 - NumPy
# 1. Generar datos ficticios de clientes y transacciones usando NumPy

np.random.seed(10)

num_clientes = 50

ids = np.arange(1, num_clientes + 1)
edades = np.random.randint(18, 70, size=num_clientes)
ciudades = np.random.choice([
    "Buenos Aires", "Córdoba", "Rosario", "Mendoza", "La Plata",
    "Salta", "Tucumán", "Santa Fe", "Neuquén", "Bahía Blanca"
], size=num_clientes)

# Total compras entre 0 y 20
compras = np.random.randint(0, 21, size=num_clientes)
# Monto promedio de compra entre 100 y 800
monto_promedio = np.random.uniform(100, 800, size=num_clientes)
# Monto total aproximado
monto_total = compras * monto_promedio

# Crear array base
clientes_numpy = np.column_stack([ids, edades, compras, monto_total])

# Operaciones básicas
cantidad_clientes = clientes_numpy.shape[0]
edad_media = edades.mean()
compras_totales = compras.sum()

cantidad_clientes, edad_media, compras_totales


Celda 3 – Guardar datos NumPy

In [ ]:
# Guardar datos en archivo .npy para la siguiente lección
np.save("clientes_numpy.npy", clientes_numpy)
"Archivo clientes_numpy.npy guardado."


Celda 4 – Lección 2: cargar NumPy en Pandas

In [ ]:
# LECCIÓN 2 - Pandas desde NumPy

# 1. Leer los datos preparados en NumPy y convertirlos en DataFrame
clientes_numpy_cargado = np.load("clientes_numpy.npy")

df_numpy = pd.DataFrame(
    clientes_numpy_cargado,
    columns=["ID", "Edad", "Total_Compras", "Monto_Total"]
)

df_numpy.head()


Celda 5 – Exploración con Pandas

In [ ]:
# Exploración inicial

display(df_numpy.head())
display(df_numpy.tail())
display(df_numpy.describe())

# Filtros de ejemplo
mayores_30 = df_numpy[df_numpy["Edad"] > 30]
con_mas_de_5_compras = df_numpy[df_numpy["Total_Compras"] > 5]

mayores_30.head(), con_mas_de_5_compras.head()


Celda 6 – Guardar CSV preliminar

In [ ]:
# Guardar DataFrame preliminar para la Lección 3
df_numpy.to_csv("clientes_desde_numpy.csv", index=False)
"Archivo clientes_desde_numpy.csv guardado."


Celda 7 – Lección 3: obtención de datos (CSV, Excel, web)

In [ ]:
# LECCIÓN 3 - Obtención de datos desde archivos

# 1. Cargar el CSV generado en la Lección 2
df_base = pd.read_csv("clientes_desde_numpy.csv")

# 2. Incorporar nuevas fuentes de datos: tus archivos de e-commerce
df_ecom_csv = pd.read_csv("clientes_ecommerce.csv")
df_ecom_xlsx = pd.read_excel("clientes_ecommerce.xlsx")

display(df_base.head())
display(df_ecom_csv.head())
display(df_ecom_xlsx.head())


Celda 8 – Lectura opcional desde web

In [ ]:
# 3. Opcional: leer una tabla desde la web (puedes omitir si no tienes conexión)

try:
    tablas_web = pd.read_html(
        "https://en.wikipedia.org/wiki/List_of_cities_in_Argentina_by_population"
    )
    tabla_ciudades = tablas_web[0]
    display(tabla_ciudades.head())
except Exception as e:
    tabla_ciudades = pd.DataFrame()
    print("No se pudo leer la tabla web:", e)


Celda 9 – Unificación de fuentes

In [ ]:
# 4. Unificar fuentes de datos

df_ecom_csv["ID"] = df_ecom_csv["ID"].astype(int)
df_base["ID"] = df_base["ID"].astype(int)

df_unificado = pd.merge(
    df_base,
    df_ecom_csv[["ID", "Nombre", "Ciudad"]],
    on="ID",
    how="left"
)

display(df_unificado.head())
df_unificado.to_csv("dataset_consolidado.csv", index=False)
"Archivo dataset_consolidado.csv guardado."


Celda 10 – Lección 4: nulos y outliers

In [ ]:
# LECCIÓN 4 - Manejo de valores perdidos y outliers

# 1. Identificar valores nulos
nulos_por_columna = df_unificado.isna().sum()
nulos_por_columna


Celda 11 – Imputación y outliers

In [ ]:
# 2. Imputar Edad faltante con la media
if "Edad" in df_unificado.columns:
    df_unificado["Edad"] = df_unificado["Edad"].fillna(df_unificado["Edad"].mean())

# 3. Detectar outliers en Monto_Total usando IQR
q1 = df_unificado["Monto_Total"].quantile(0.25)
q3 = df_unificado["Monto_Total"].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

outliers = df_unificado[
    (df_unificado["Monto_Total"] < limite_inferior)
    | (df_unificado["Monto_Total"] > limite_superior)
]

outliers.head()


Celda 12 – Tratamiento de outliers y guardado

In [ ]:
# Recortar Monto_Total al rango permitido
df_unificado["Monto_Total"] = df_unificado["Monto_Total"].clip(
    lower=limite_inferior,
    upper=limite_superior
)

df_limpio = df_unificado.copy()
df_limpio.to_csv("dataset_limpio.csv", index=False)
"Archivo dataset_limpio.csv guardado."


Celda 13 – Lección 5: Data Wrangling

In [ ]:
# LECCIÓN 5 - Data Wrangling

# 1. Tomar DataFrame limpio y simular duplicados
df_wrangle = pd.concat([df_limpio, df_limpio.iloc[:3]], ignore_index=True)

# 2. Eliminar registros duplicados
df_wrangle = df_wrangle.drop_duplicates()

# 3. Transformar tipos de datos
if "Ciudad" in df_wrangle.columns:
    df_wrangle["Ciudad"] = df_wrangle["Ciudad"].astype("category")

df_wrangle.head()


Celda 14 – Nuevas columnas, segmentación y normalización

In [ ]:
# 4. Crear nuevas columnas calculadas

# Ticket promedio
df_wrangle["Ticket_Promedio"] = df_wrangle.apply(
    lambda row: row["Monto_Total"] / row["Total_Compras"] if row["Total_Compras"] > 0 else 0,
    axis=1
)

# Segmento por monto total
bins = [0, 1000, 3000, 10000]
labels = ["Bajo", "Medio", "Alto"]
df_wrangle["Segmento_Monto"] = pd.cut(
    df_wrangle["Monto_Total"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Normalización min-max de Monto_Total
min_monto = df_wrangle["Monto_Total"].min()
max_monto = df_wrangle["Monto_Total"].max()

if max_monto > min_monto:
    df_wrangle["Monto_Normalizado"] = (
        df_wrangle["Monto_Total"] - min_monto
    ) / (max_monto - min_monto)
else:
    df_wrangle["Monto_Normalizado"] = 0

display(df_wrangle.head())
df_wrangle.to_csv("dataset_wrangle.csv", index=False)
"Archivo dataset_wrangle.csv guardado."


Celda 15 – Lección 6: agrupamiento

In [ ]:
# LECCIÓN 6 - Agrupamiento y pivoteo de datos

# 1. Agrupamiento por ciudad
if "Ciudad" in df_wrangle.columns:
    agrupado_ciudad = df_wrangle.groupby("Ciudad").agg({
        "Total_Compras": "sum",
        "Monto_Total": ["sum", "mean"],
        "Ticket_Promedio": "mean"
    })
    display(agrupado_ciudad)
else:
    agrupado_ciudad = pd.DataFrame()
    agrupado_ciudad


Celda 16 – Pivot y melt

In [ ]:
# 2. Pivot por Ciudad y Segmento_Monto
if "Ciudad" in df_wrangle.columns and "Segmento_Monto" in df_wrangle.columns:
    tabla_pivot = df_wrangle.pivot_table(
        index="Ciudad",
        columns="Segmento_Monto",
        values="Monto_Total",
        aggfunc="mean"
    )
    display(tabla_pivot)
else:
    tabla_pivot = pd.DataFrame()

# 3. Melt: volver a formato largo
if not tabla_pivot.empty:
    tabla_melt = tabla_pivot.reset_index().melt(
        id_vars="Ciudad",
        var_name="Segmento_Monto",
        value_name="Monto_Promedio"
    )
    display(tabla_melt.head())
else:
    tabla_melt = pd.DataFrame()


Celda 17 – Exportar dataset final

In [ ]:
# 4. Exportar dataset final listo para análisis

df_wrangle.to_csv("dataset_final.csv", index=False)
df_wrangle.to_excel("dataset_final.xlsx", index=False)

"dataset_final.csv y dataset_final.xlsx exportados correctamente."
